In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### Type 1 MCP Server: **Local**

In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/shre.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None, title='Create new entities with observations'),
 Tool(name='search_nodes', description='Search for entities and their relations using text search with relevance ranking', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'query': {'type': 'string'}, 'limit': {'type': 'number'}}, 'required': ['query']}, annotations=None, title='Search for entities and their relations using text search with relevance ranking'),
 Tool(name='read_graph', description='Get recent entit

In [6]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Shreyash. I'm an aspiring AI and LLM engineer. I'm learning a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

In [7]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Shreyash! It's great that you're diving into AI Agents and learning about the MCP protocol. If you want, I can help you summarize what you've learned, answer questions, or keep track of your learning progress. How would you like to proceed?

In [8]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Shreyash. What do you know about me?")
    display(Markdown(result.final_output))

I know that you are an aspiring AI and LLM engineer. Is there anything else you'd like to share or ask about?

Check the traces: https://platform.openai.com/traces

### Type 2 MCP Server: **Local but uses API over the internet**

In [10]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [11]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4.1-mini"

In [12]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The latest news on Amazon stock (ticker AMZN) indicates that as of 11/25/2025, Amazon's share price closed at $229.59, showing a slight drop of about 0.47% from the opening price that day. Amazon's market capitalization is approximately $1.97 trillion. Recent reports also highlight Amazon's third-quarter results, which beat the high end of guidance on revenue, although net income was impacted by $4.3 billion in unusual items. Overall, this points to a solid top-line performance with some bottom-line challenges due to extraordinary expenses. The stock outlook appears cautiously optimistic given strong revenue but with some profitability pressure.

### Type 3 MCP Server: **Remote/Hosted/Managed**

Not a common type of MCP server since most of them are paid versions at high costs and usually hosted in big companies, meant for other companies.

### Back to Type 2 MCP Server

Polygon.io is a hugely popular financial data provider. 

In [13]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [14]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=278.85, high=279, low=275.9865, open=277.26, timestamp=1764363600000, volume=20100859.0, vwap=277.63)

To avoid hitting rate limits on the free plan, there is a function devised in the python script `market.py` which checks if the plan is free or not (as defined in the environment variable `POLYGON_PLAN`) and if it is not paid, it gets the price for the full day and caches it.

In [15]:
from market import get_share_price
get_share_price("AAPL")

278.85

In [16]:
for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

278.85

Creating an MCP server for this operation.

In [18]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

In [19]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $278.85.